In [1]:
# Storage SC

In [2]:
import logging

import smartpynector as sp
from utils import *

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Constants
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
}

MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

COMMAND_GRAPH_PATTERN = """?command rdf:type saref:GetMeterHistoryCommand .
                        ?command saref:isCommandOf ?device_id .
                        ?command saref:hasTimestampStart ?startTimestamp .
                        ?command saref:hasTimestampEnd ?endTimestamp .
                        ?command saref:relatesToProperty saref:Temperature ."""

In [5]:
# Return last 100 measurements from GraphDB
def handle_answer_measurements(query_bindings):
    for binding in query_bindings:
        logger.info(f"Answer query bindings: {binding}")
        start_timestamp = binding["startTimestamp"]
        end_timestamp = binding["endTimestamp"]
        sparql_query = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                            PREFIX saref: <https://w3id.org/saref#>

                            SELECT *
                            WHERE {{
                                ?meas rdf:type saref:Measurement .
                                ?meas saref:hasValue ?temp .
                                ?meas saref:isMeasuredIn saref:TemperatureUnit .
                                ?meas saref:hasTimestamp ?timestamp .
                                FILTER (?timestamp >= "{start_timestamp}"^^xsd:date && ?timestamp <= "{end_timestamp}"^^xsd:date)
                                ?meas saref:isMeasurementOf ?room_id .
                                ?meas saref:relatesToProperty saref:Temperature .
                                ?meas saref:measurementMadeBy ?device_id .
                            }}
                            """
        res = sp.run_sparql_query(READ_URL, sparql_query)
        logger.info(res)

    return [res]


def handle_react_measurements(bindings):
    logger.info(f"Handling react binding: {bindings}")
    sp.store_data_in_graphdb(
        graph_pattern=MEAS_GRAPH_PATTERN,
        binding_set=bindings,
        prefixes=PREFIXES,
        read_url=READ_URL,
        write_url=WRITE_URL,
    )
    return []

In [6]:
def start_storage_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    answer_measurements_ki = register_answer_knowledge_interaction(
        COMMAND_GRAPH_PATTERN,
        "answer-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    react_measurements_ki = register_react_knowledge_interaction(
        MEAS_GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    start_handle_loop(
        {
            answer_measurements_ki: handle_answer_measurements,
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )

In [7]:
start_storage_kb(
    "http://example.org/storage",
    "Storage",
    "GraphDB smart connector",
    "http://knowledge_engine:8280/rest/",
)

2023-06-20 01:27:34 INFO registered Storage
2023-06-20 01:27:34 INFO received issued knowledge interaction id: http://example.org/storage/interaction/answer-measurements
2023-06-20 01:27:34 INFO received issued knowledge interaction id: http://example.org/storage/interaction/react-measurements
2023-06-20 01:28:28 INFO Answer query bindings: {'device_id': '"http://0.0.0.0:8001/thermostat/devices/1"', 'endTimestamp': '2023-06-20 01:33:00+00:00', 'startTimestamp': '2023-06-19 09:01:00+00:00', 'command': '<http://0.0.0.0:8001/thermostat/command/92851539-5b1b-4548-9bb0-57d4330d8dba>'}
2023-06-20 01:28:28 INFO {'head': {'vars': ['meas', 'temp', 'timestamp', 'room_id', 'device_id']}, 'results': {'bindings': []}}


AssertionError: 